In [4]:
import cv2 
import numpy as np 
import mediapipe as mp
from tensorflow.keras.models import load_model



In [5]:
#iniltilize mediapipe
mp_hands = mp.solutions.hands
hands =mp_hands.Hands(max_num_hands=1,min_detection_confidence=0.7)
mp_draw = mp.solutions.drawing_utils

In [6]:
#load the gesture recognizer model
model = load_model("mp_hand_gesture")

In [7]:
#load class names 
f = open('gesture.names', 'r')
labels = f.read().split('\n')
f.close()
print(labels)

['okay', 'peace', 'thumbs up', 'thumbs down', 'call me', 'stop', 'rock', 'live long', 'fist', 'smile']


In [8]:

# initialize the webcam
cam = cv2.VideoCapture(0)

while True:
    # read each frame from the webcam
    _, frame = cam.read()
    x, y, c = frame.shape
    # flip the frame vertically
    frame = cv2.flip(frame, 1)
    framergb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    # get hand landmark prediction
    result = hands.process(framergb)
    # print(result)
    class_name = ''
    # post-process the result
    if result.multi_hand_landmarks:
        for handslms in result.multi_hand_landmarks:
            landmarks = []
            for lm in handslms.landmark:  # Use 'landmark' instead of 'landmarks'
                lmx = int(lm.x * x)
                lmy = int(lm.y * y)
                landmarks.append([lmx, lmy])
            # drawing landmarks on frame
            mp_draw.draw_landmarks(frame, handslms, mp_hands.HAND_CONNECTIONS)
            # predict gesture
            prediction = model.predict([landmarks])
            # print prediction
            classID = np.argmax(prediction)
            class_name = labels[classID].capitalize()
    #show the predictions on the frame 
    cv2.putText(frame, class_name,(10,50),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),2)
    #show the final output
    cv2.imshow('Output', frame)
    if cv2.waitKey(1) == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 46ms/step


KeyboardInterrupt: 